In [1]:
#let's import

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans

!pip install folium
import folium

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

print("imports is DONE")

imports is DONE


In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
post_codes = requests.get(wiki_url).text

In [9]:
wiki_xml = BeautifulSoup(post_codes, 'xml')
wiki_table=wiki_xml.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [10]:
for tr_cell in wiki_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [11]:
df

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [12]:
df=df[df['Borough']!='Not assigned']

In [13]:
df2=df[df['Neighborhood']=='Not assigned']
df2

,Postalcode,Borough,Neighborhood


##### There is no record with Neigborhood = "Not assigned", so nothing to do.

In [14]:
df

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_merged'},inplace=True)

    Postalcode                                Neighborhood_merged
0          M1B                                     Malvern, Rouge
1          M1C             Rouge Hill, Port Union, Highland Creek
2          M1E                  Guildwood, Morningside, West Hill
3          M1G                                             Woburn
4          M1H                                          Cedarbrae
..         ...                                                ...
98         M9N                                             Weston
99         M9P                                          Westmount
100        M9R  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W                Northwest, West Humber - Clairville

[103 rows x 2 columns]


In [17]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [18]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [19]:
df_merge.drop_duplicates(inplace=True)

In [20]:
df_merge.rename(columns={'Neighborhood_merged':'Neighborhood'},inplace=True)

In [21]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df_merge.shape

(103, 3)

# Q2

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [23]:
geocode_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [24]:
geocode_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
geocode_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geocode_merged = pd.merge(geocode_df, df_merge, on='Postalcode')

In [26]:
geocode_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [27]:
geocode_final = geocode_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geocode_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Q3

In [28]:
geo_toronto=geocode_final[geocode_final['Borough'].str.contains("Toronto")]
geo_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [29]:
client_id     = '2QFOBVAGM03NBKT2O4UKAKZQG1HXC1UKBVZKOJNKCLNPHGY1' # Foursquare ID
client_secret = 'XNVPMAN5T5TN2LJWSQY1IW1WAOISTHKXU0HHIMLO1ZNG5WJV' # Foursquare Secret
version       = '20201126'                                         # today

In [30]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    limit=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
nearby_venues = getNearbyVenues(names=geo_toronto['Neighborhood'],
                                latitudes=geo_toronto['Latitude'],
                                longitudes=geo_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [32]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [33]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [34]:
# one hot encoding
nearby_venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
nearby_venues_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
nearby_venues_onehot.insert(loc=0, column='Neighborhood', value=nearby_venues['Neighborhood'] )
nearby_venues_onehot.shape

(1639, 240)

In [35]:
nearby_venues_grouped = nearby_venues_onehot.groupby('Neighborhood').mean().reset_index()
nearby_venues_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.0,0.014706


In [37]:
def top_venues(row, num_of_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_of_venues]

In [40]:
num_of_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_of_venues):
    try:
        columns.append('{}{} Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Venue'.format(ind+1))

top_venues_sorted = pd.DataFrame(columns=columns)
top_venues_sorted['Neighborhood'] = nearby_venues_grouped['Neighborhood']

for ind in np.arange(nearby_venues_grouped.shape[0]):
    top_venues_sorted.iloc[ind, 1:] = top_venues(nearby_venues_grouped.iloc[ind, :], num_of_venues)

top_venues_sorted.head()

,Neighborhood,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
0,Berczy Park,Coffee Shop,Restaurant,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Cheese Shop,Concert Hall,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Gym,Playground,Stadium,Bar,Bakery,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Brewery,Garden,Restaurant,Fast Food Restaurant,Farmers Market,Auto Workshop,Butcher,Burrito Place,Recording Studio,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Bar,Boat or Ferry,Historic Site,Sculpture Garden,Rental Car Location,Harbor / Marina,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Thai Restaurant,Bubble Tea Shop,Japanese Restaurant,Salad Place,Department Store


In [51]:
# set number of clusters
num_of_clusters = 5

nearby_venues_grouped_clustering = nearby_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
k_means = KMeans(n_clusters=num_of_clusters, random_state=0).fit(nearby_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
k_means.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [53]:
# add clustering labels
top_venues_sorted.insert(0, 'Cluster Labels', k_means.labels_)

geo_toronto_merged = geo_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
geo_toronto_merged = geo_toronto_merged.join(top_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

geo_toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Trail,Yoga Studio,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Caribbean Restaurant,Pub
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Pet Store,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pizza Place,Coffee Shop,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [56]:
top_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Cheese Shop,Concert Hall,Bistro
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Gym,Playground,Stadium,Bar,Bakery,Italian Restaurant
2,0,"Business reply mail Processing Centre, South C...",Brewery,Garden,Restaurant,Fast Food Restaurant,Farmers Market,Auto Workshop,Butcher,Burrito Place,Recording Studio,Pizza Place
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Bar,Boat or Ferry,Historic Site,Sculpture Garden,Rental Car Location,Harbor / Marina,Coffee Shop
4,0,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Thai Restaurant,Bubble Tea Shop,Japanese Restaurant,Salad Place,Department Store


In [60]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# color scheme for the clusters
import matplotlib.colors as colors
x = np.arange(num_of_clusters)
ys = [i + x + (i*x)**2 for i in range(num_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# markers on the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        geo_toronto_merged['Latitude'], 
        geo_toronto_merged['Longitude'], 
        geo_toronto_merged['Neighborhood'], 
        geo_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
#Cluster-1:

geo_toronto_merged.loc[geo_toronto_merged['Cluster Labels'] == 0, geo_toronto_merged.columns[[1] + list(range(5, geo_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Caribbean Restaurant,Pub
42,East Toronto,0,Park,Pet Store,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pizza Place,Coffee Shop,Sushi Restaurant
43,East Toronto,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
45,Central Toronto,0,Department Store,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Breakfast Spot,Gym / Fitness Center,Park,Donut Shop
46,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Salon / Barbershop,Spa,Fast Food Restaurant,Sporting Goods Shop,Diner,Restaurant
47,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Toy / Game Store,Farmers Market
49,Central Toronto,0,Coffee Shop,Sushi Restaurant,American Restaurant,Supermarket,Bank,Fried Chicken Joint,Restaurant,Bagel Shop,Pizza Place,Liquor Store
51,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Park,Chinese Restaurant,Italian Restaurant,Pub,Market
52,Downtown Toronto,0,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Restaurant,Café,Hotel,Men's Store,Yoga Studio,Bubble Tea Shop
53,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant


In [62]:
# Cluster-2

geo_toronto_merged.loc[geo_toronto_merged['Cluster Labels'] == 1, geo_toronto_merged.columns[[1] + list(range(5, geo_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
37,East Toronto,1,Pub,Health Food Store,Trail,Yoga Studio,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
50,Downtown Toronto,1,Park,Tennis Court,Playground,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
63,Central Toronto,1,Dive Bar,Music Venue,Garden,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
64,Central Toronto,1,Sushi Restaurant,Park,Mexican Restaurant,Jewelry Store,Trail,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [63]:
# Cluster-3

geo_toronto_merged.loc[geo_toronto_merged['Cluster Labels'] == 2, geo_toronto_merged.columns[[1] + list(range(5, geo_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
48,Central Toronto,2,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [64]:
# Cluster-4

geo_toronto_merged.loc[geo_toronto_merged['Cluster Labels'] == 3, geo_toronto_merged.columns[[1] + list(range(5, geo_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
44,Central Toronto,3,Park,Bus Line,Swim School,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
